In [ ]:
import io
import cv2
import base64
import numpy as np
from PIL import Image
from pathlib import Path
from IPython.display import HTML
from typing import Any, Optional, Union


def display_images_grid_html(images, cols=3, row_height=200, labels=None, font_scale=1.0):
    
    def _convert_imageType_for_html(image: Any) -> Optional[Union[Path, str]]:
        
        if isinstance(image, (str, Path)):
            path = Path(image)
            if path.is_file() and path.exists(): return path
            else: return None

        if isinstance(image, Image.Image):
            buffered = io.BytesIO()
            image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
            return f"data:image/png;base64,{img_base64}"

        if isinstance(image, np.ndarray):
            ret, buf = cv2.imencode('.png', image)
            if ret:
                img_base64 = base64.b64encode(buf.tobytes()).decode("utf-8")
                return f"data:image/png;base64,{img_base64}"
            else:
                return None

        return None
    
    if labels is not None and len(labels) != len(images):
        raise ValueError("labels の要素数は images の要素数と一致する必要があります")
    
    # フォントサイズの設定（基本サイズ 14px に倍率をかける）
    base_font_size = 14
    font_size = base_font_size * font_scale

    html = "<table style='border-collapse: collapse;'>"
    for i, img in enumerate(images):
        if i % cols == 0:
            html += "<tr>"
        
        converted = _convert_imageType_for_html(img)
        if converted is None:
            converted = ""
        
        cell_content = f"<img src='{converted}' height='{row_height}px'>"
        
        if labels is not None:
            label_text = labels[i].replace("\n", "<br>")
            cell_content += f"<br><div style='text-align: center; font-size: {font_size}px;'>{label_text}</div>"
        
        html += f"<td style='padding: 5px; text-align: center;'>{cell_content}</td>"
        
        if i % cols == cols - 1:
            html += "</tr>"
            
    if len(images) % cols:
        html += "</tr>"
        
    html += "</table>"
    return HTML(html)


### 様々な形式で表示可能

In [38]:
import random
from pathlib import Path

sample_num = 10
image_paths = random.sample(list(Path("../../sample_data/coco_sample_datasets/sample_coco_train2017/").glob("*.jpg")), sample_num)

images  = [str(img) for img in image_paths[::4]] # 画像ファイルパス(str)のリスト
images += [img for img in image_paths[1::4]] # 画像ファイルパス(pathlib.Path)のリスト
images += [cv2.imread(str(img)) for img in image_paths[2::4]] # OpenCV 画像(numpy.ndarray)のリスト
images += [Image.open(img) for img in image_paths[3::4]] # Pillow 画像(PIL.Image.Image)のリスト
labels = [f"Image {i+1}\n({image_paths[i].name})" for i in range(sample_num)]

display_images_grid_html(image_paths, cols=6, row_height=200, labels=labels, font_scale=1.0)